In [35]:
import os
import cv2
import numpy as np
from keras_facenet import FaceNet
from ultralytics import YOLO

# ==== 1. Load models ====
detector = YOLO("Model/yolov8n-face.pt")  # YOLOv8n-face detector
embedder = FaceNet()                      # FaceNet embedder

# ==== 2. Load face embeddings database ====
EMBEDDING_DIR = "Face_Data/embeddings"
face_db = {}
for file in os.listdir(EMBEDDING_DIR):
    if file.endswith(".npy"):
        name = file.replace(".npy", "")
        emb = np.load(os.path.join(EMBEDDING_DIR, file))
        face_db[name] = emb

# ==== 3. Cosine similarity ====
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# ==== 4. Nhận diện khuôn mặt từ frame ====
def process_frame(img, threshold=0.6):
    results = detector.predict(source=img, conf=0.3, verbose=False)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            face_crop = img[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue

            # Nhúng khuôn mặt
            face_crop = cv2.resize(face_crop, (160, 160))
            face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
            embedding = embedder.embeddings([face_rgb])[0]

            # So sánh với database
            best_match = "Unknown"
            best_score = -1
            for name, db_emb in face_db.items():
                score = cosine_similarity(embedding, db_emb)
                if score > best_score:
                    best_score = score
                    best_match = name
            # In kết quả
            if best_score >= threshold:
                print(f" Nhận diện: {best_match} (score: {best_score:.2f})")
            else:
                print(f" Không xác định được người (score cao nhất: {best_score:.2f})")

            # Vẽ kết quả lên ảnh
            label = f"{best_match} ({best_score:.2f})" if best_score >= threshold else "Unknown"
            color = (0, 255, 0) if best_score >= threshold else (0, 0, 255)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(img, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return img


In [36]:
MODE = "image"  

    # Test ảnh
image_path = "Test/test.jpg"  
img = cv2.imread(image_path)
if img is None:
    print(" Không đọc được ảnh.")
else:
    img = process_frame(img)
    cv2.imshow("Result", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
 Nhận diện: MTP (score: 0.81)


In [38]:
MODE = "webcam"
    # Test webcam
cap = cv2.VideoCapture(0)
print(" Đang mở webcam... Nhấn Q để thoát.")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = process_frame(frame)
    cv2.imshow("Webcam Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


 Đang mở webcam... Nhấn Q để thoát.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
 Nhận diện: MTP (score: 0.75)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
 Nhận diện: MTP (score: 0.73)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
 Nhận diện: MTP (score: 0.74)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
 Nhận diện: MTP (score: 0.75)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
 Nhận diện: MTP (score: 0.80)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
 Nhận diện: MTP (score: 0.79)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
 Nhận diện: MTP (score: 0.75)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
 Nhận diện: MTP (score: 0.77)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
 Nhận diện: MTP (score: 0.75)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
 Không xác định được người (score cao nhất: 0.50)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
 Không xác định được người (score cao nhất: 0.45)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
 Không xác định được người (score cao nhất: 0.51)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
 Không xác định được người (score cao nhất: 0.60)
